In [2]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras

In [8]:
DATA_PATH = "datasets/processed_data/data.json"

In [9]:
def load_data(data_path):
    """Loads training dataset from json file.

        :param data_path (str): Path to json file containing data
        :return X (ndarray): Inputs
        :return y (ndarray): Targets
    """

    with open(data_path, "r") as fp:
        data = json.load(fp)

    X = np.array(data["mfcc"])
    y = np.array(data["labels"])
    return X, y

In [10]:
def prepare_datasets(test_size, validation_size):
    """Loads data and splits it into train, validation and test sets.

    :param test_size (float): Value in [0, 1] indicating percentage of data set to allocate to test split
    :param validation_size (float): Value in [0, 1] indicating percentage of train set to allocate to validation split

    :return X_train (ndarray): Input training set
    :return X_validation (ndarray): Input validation set
    :return X_test (ndarray): Input test set
    :return y_train (ndarray): Target training set
    :return y_validation (ndarray): Target validation set
    :return y_test (ndarray): Target test set
    """

    # load data
    X, y = load_data(DATA_PATH)

    # create train, validation and test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_size)

    return X_train, X_validation, X_test, y_train, y_validation, y_test


In [11]:
def build_model(input_shape):
    """Generates RNN-LSTM model

    :param input_shape (tuple): Shape of input set
    :return model: RNN-LSTM model
    """

    # build network topology
    model = keras.Sequential()

    # 2 LSTM layers
    model.add(keras.layers.LSTM(64, input_shape=input_shape, return_sequences=True))
    model.add(keras.layers.LSTM(64))

    # dense layer
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dropout(0.3))

    # output layer
    model.add(keras.layers.Dense(10, activation='softmax'))

    return model

In [12]:
# get train, validation, test splits
X_train, X_validation, X_test, y_train, y_validation, y_test = prepare_datasets(0.25, 0.2)

In [13]:
# create network
input_shape = (X_train.shape[1], X_train.shape[2]) # 130, 13
model = build_model(input_shape)

In [14]:
# compile model
optimiser = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimiser,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [15]:
model.summary

<bound method Model.summary of <keras.src.engine.sequential.Sequential object at 0x7fece7dfb460>>

In [16]:
# train model
history = model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=32, epochs=30)

Epoch 1/30
93/93 [==============================] - 39s 358ms/step - loss: 2.2600 - accuracy: 0.1452 - val_loss: 2.1894 - val_accuracy: 0.2396
Epoch 2/30
93/93 [==============================] - 30s 317ms/step - loss: 2.1430 - accuracy: 0.2466 - val_loss: 2.0704 - val_accuracy: 0.3230
Epoch 3/30
93/93 [==============================] - 33s 353ms/step - loss: 2.0383 - accuracy: 0.3032 - val_loss: 1.9600 - val_accuracy: 0.3553
Epoch 4/30
93/93 [==============================] - 32s 341ms/step - loss: 1.9437 - accuracy: 0.3309 - val_loss: 1.8502 - val_accuracy: 0.3742
Epoch 5/30
93/93 [==============================] - 30s 325ms/step - loss: 1.8521 - accuracy: 0.3598 - val_loss: 1.7780 - val_accuracy: 0.3863
Epoch 6/30
93/93 [==============================] - 31s 328ms/step - loss: 1.7696 - accuracy: 0.3972 - val_loss: 1.7115 - val_accuracy: 0.4078
Epoch 7/30
93/93 [==============================] - 31s 337ms/step - loss: 1.7028 - accuracy: 0.4195 - val_loss: 1.6564 - val_accuracy: 0.4240

In [17]:
# evaluate model on test set
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print('\nTest accuracy:', test_acc)

39/39 - 3s - loss: 1.3201 - accuracy: 0.5441 - 3s/epoch - 68ms/step

Test accuracy: 0.5440582036972046
